In [1]:
import importlib

In [2]:
from pyvista import imred, tv, spectra, stars, slitmask, image
import numpy as np
import pdb
import copy
import matplotlib.pyplot as plt
import os
from astropy.table import vstack
import pandas as pd

In [3]:
# Use these lines if you are running the notebook yourself. Matplotlib
# window will open outside the notebook, which is the desired behavior so
# you can have a single display tool, which you should leave open. Other
# plot windows will also appear outside the notebook, which you can close
# as desired
%matplotlib qt
t=tv.TV()
plotinter=True

# following lines only for fully non-interactive demo of notebook
#%matplotlib inline
#plotinter=False
#t=None

In [4]:
# put directory name with images here
indir='UT230909'
red=imred.Reducer('KOSMOS',dir=indir)

INSTRUMENT: KOSMOS   config: 
  will use format:  UT230909/*{:04d}.f*.fits*
         gain:  [0.6]    rn: [5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


In [5]:
red.log().show_in_notebook(display_length=10)

idx,FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
0,Flat_SEG3R1.0008.fits,2023-09-09T01:47:35.037138,,6:56:00.00,75:00:00.00,0.5
1,Flat_SEG3G2.0009.fits,2023-09-09T01:49:32.885458,,6:56:00.00,75:00:00.00,0.5
2,SEG3G2.0011.fits,2023-09-09T02:33:52.513049,Seg3G2-Mask,21:21:34.14,19:09:06.69,20.0
3,SEG3G2.0012.fits,2023-09-09T02:36:34.405725,Seg3G2-Mask,21:21:34.14,19:09:06.69,11.2
4,SEG3G2.0015.fits,2023-09-09T02:59:25.399982,Seg3G2-Mask,21:21:33.90,19:09:09.95,20.0
5,SEG3G2.0016.fits,2023-09-09T03:02:17.891705,Seg3G2-Mask,21:21:33.90,19:09:09.95,300.0
6,SEG3G2.0017.fits,2023-09-09T03:09:16.686771,Seg3G2-Mask,21:21:33.79,19:09:08.51,1200.0
7,SEG3G2.0020.fits,2023-09-09T03:54:34.957512,Seg3G2-Mask,21:21:33.86,19:09:09.37,1200.0
8,Flat_SEG3G2.0021.fits,2023-09-09T04:15:36.148095,Seg3G2-Mask,21:21:33.86,19:09:09.37,2.0
9,Flat_SEG3G2.0022.fits,2023-09-09T04:17:04.730402,Seg3G2-Mask,21:21:33.86,19:09:09.37,2.0


In [6]:
flatims=[22]
flat=red.mkflat(flatims,spec=True,display=None,littrow=False,)

  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [7]:
if t is not None:
    t.tv(flat)

In [8]:
biastims=[74,75,76,77,78]
bias=red.mkbias(biastims,display=None)

  Reading file: UT230909\Bias.0074.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0075.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0076.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0077.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0078.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [9]:
if t is not None:
    t.tv(bias)

In [10]:
darktims=[94,96,96]
dark=red.mkdark(darktims,display=None)

  Reading file: UT230909\dark.0094.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0096.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0096.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [11]:
if t is not None:
    t.tv(dark)

In [12]:
star1=red.reduce(20, crbox='lacosmic', bias=bias, display=t)

  Reading file: UT230909\SEG3G2.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  See bias box (solid outlines applied to dashed regions of the same color), and cross section. 
   To continue, hit space in display window (p for debug) 
  subtracting bias...
  zapping CRs with astroscrappy detect_cosmics
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [13]:
flat1 = red.reduce(22)

  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [14]:
# this does not take mkflat().
trace=spectra.Trace(transpose=True)
t.tvclear()
bottom,top = trace.findslits(flat1,display=t,thresh=0.5,sn=True)

In [15]:
vars(trace)

{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'model': [Polynomial([205.39570053,   6.70214955, -19.58890376], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([353.47561566,   6.31656815, -13.19157469], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([500.31128681,   6.02780702,  -9.3204817 ], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([725.38596237,   6.39430102,  -3.60916434], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([934.65809011,   6.48687832,   1.55820243], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1023.40503861,    6.46322524,    3.67504815], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1089.0796024,    6.4892661,    5.2677399], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1154.51935672,    6.50896277,    6.81061458], d

In [16]:
trace.rows

[[205, 221],
 [353, 369],
 [500, 516],
 [725, 764],
 [935, 973],
 [1023, 1062],
 [1089, 1128],
 [1154, 1193],
 [1307, 1323],
 [1352, 1391],
 [1436, 1475],
 [1542, 1581],
 [1615, 1654],
 [1681, 1720],
 [1735, 1751],
 [1763, 1802]]

In [17]:
kmsfile1='kms/kosmos.23.seg3g2Copy_2.kms'
targets1 = slitmask.read_kms(kmsfile1, sort='YMM')# sort='YMM'

In [18]:
df = targets1.to_pandas()

In [19]:
df # A look at your table in panda formate

,ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
0,TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
1,TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
2,TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
3,TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.870,-10.801
4,TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.610
5,TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
6,TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
7,TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
8,TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272
9,TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.780


In [20]:
# Specify the indices of the rows you want to remove
rows_to_remove = [0,1,2,3,4,5,8,10,12,14,15]

# Remove the specified rows
df_cleaned = df.drop(rows_to_remove)

In [21]:
# Give your index value
in_dex = [6, 7, 9, 11, 13]

# Create a new list to store the filtered lines
filtered_rows = []

for index, line in enumerate(trace.rows):
    if index in in_dex:
        filtered_rows.append(line)

# Replace the original trace.rows with the filtered list
trace.rows = filtered_rows

In [22]:
# Create your own trace from scratch.
trace1=spectra.Trace(sc0=2048,lags=range(-100,100),
                    rows= trace.rows ,transpose=red.transpose, rad=5, degree= 3) #[1585,1545],# 1372 #1545-1585
vars(trace1)

{'type': 'Polynomial1D',
 'degree': 3,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-100, 100),
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]],
 'model': None,
 'sigmodel': None,
 'sc0': 2048}

In [23]:
#Trace
srow= [1110, 1173, 1372,1564,1697]   #1173,746,954,1041, # list to allow for multiple spectra on an image, manually set
#srow,ids=trace.findpeak(crstar, thresh=50)  # alternatively, find peak(s)
# trace.find(star) will find the highest peak by cross-correlation
# trace.find(star,inter=True,display=t)  will let you mark a trace location

# rad is setting the width of your trace. It will take the center position to be the star position given taking from DS9.
trace1.trace(star1,srow,skip=10,
                    gaussian = True, display=t, rad= 5)
vars(trace1)

C:\Users\abdul\pyvista\python\pyvista\spectra.py:1200: RuntimeWarning: invalid value encountered in divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
C:\Users\abdul\pyvista\python\pyvista\spectra.py:1205: RuntimeWarning: invalid value encountered in divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<rad))[0]


  Tracing row: 1697
  See trace. Hit space bar to continue....


{'type': 'Polynomial1D',
 'degree': 3,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': array([ 0.        , 29.5       , 17.        , ..., 29.00000191,
        12.        , 29.00000191]),
 'rad': 5,
 'transpose': True,
 'lags': range(-100, 100),
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]],
 'model': [<Polynomial1D(3, c0=1106.39405868, c1=-0.00055783, c2=0.00000087, c3=0.)>,
  <Polynomial1D(3, c0=1170.49549057, c1=-0.0017412, c2=0.00000102, c3=0.)>,
  <Polynomial1D(3, c0=1374.62537593, c1=-0.0069165, c2=0.00000232, c3=0.)>,
  <Polynomial1D(3, c0=1572.10212887, c1=-0.01205557, c2=0.00000354, c3=0.)>,
  <Polynomial1D(3, c0=1709.76288931, c1=-0.01565263, c2=0.00000437, c3=0.)>],
 'sigmodel': [<Polynomial1D(0, c0=2.36324237)>,
  <Polynomial1D(0, c0=2.35713388)>,
  <Polynomial1D(0, c0=2.27991788)>,
  <Polynomial1D(0, c0=2.28479207)>,
  <Polynomial1D(0, c0=2.16576251)>],
 'sc0': 2048,
 'spectrum2d': array([[-17.        ,   9.5       ,  32.        , ..., -1

In [24]:
#Frame 15 is He, 16 is Ne, and 17 is Ar
arcs=red.sum([23,])
if t is not None:
    t.clear()
    t.tv(arcs)

  Reading file: UT230909\Ne_120s_SEG3G2.0023.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  combining data with sum....
  calculating uncertainty....


In [25]:
arcec1=trace1.extract2d(arcs, display=t)

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


In [26]:
if len(targets1) == len(bottom) : 
    for arc,target in zip(arcec1,targets1) :
        arc.header['XMM'] = target['XMM']
        arc.header['YMM'] = target['YMM']
else :
    print('ERROR, number of identified slits does not match number of targets')

In [28]:
for i,arc in enumerate(arcec1) :
    
    wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
    nrow=arc.shape[0] # this is referring to the slit width of each slit. 
    # get initial guess at shift from reference using XMM (KOSMOS red low!)
    shift=int(arc.header['XMM'])#*-22.5) # 500 #-wav.pix0)
    lags=np.arange(shift-400,shift+400)

    iter = True
    while iter :
        iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,
                            lags=lags,thresh=10,file='new_wave_lamps/new_neon_red_center.dat')
        lags=np.arange(-150,150)
        plt.close()
        
    # Do the 2D wavelength solution, sampling 5 locations across slitlet
    wav.identify(arc,plot=True,nskip=nrow//5,thresh=10)
    plt.close()

  rms:    0.177 Angstroms (50 lines)
  cross correlating with reference spectrum using lags between:  -405 394
  Derived pixel shift from input wcal:  [-255.42885434]
  See identified lines.
  rms:    2.637 Angstroms (24 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    2.637 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -300 299
  Derived pixel shift from input wcal for row: 38 0


C:\Users\abdul\pyvista\python\pyvista\spectra.py:695: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:  118.139
rejecting 3 points from 141 total: 
  rms:    1.142
rejecting 3 points from 141 total: 
  See 2D wavecal fit. Enter space in plot window to continue

  rms:    0.177 Angstroms (50 lines)
  cross correlating with reference spectrum using lags between:  -421 378
  Derived pixel shift from input wcal:  [99.11211696]
  See identified lines.
  rms:    1.405 Angstroms (27 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    1.405 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -300 299
  Derived pixel shift from input wcal for row: 38 0
  See identified lines.
  rms:    0.162
rejecting 5 points from 155 total: 
  rms:    0.094
rejecting 5 points from 155 total: 
  Se

In [29]:
# Sky window starts right on top of spectra window.
starec2=trace1.extract2d(star1, display= t,) # rad = 3 back=[[9,3],[-9,-3]],

#plt.figure()
#plt.plot(starec2.data[0])
#plt.plot(starec2.data[0])
#vars(starec2)

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


In [30]:
t.clear()
plt.figure()
for i,(o,a) in enumerate(zip(starec2,arcec1)) :
    print(o.shape)
    print(a.wave)
    o.add_wave(a.wave)
    name = o.header["FILE"].split(".")[0]
    print(name)
    #t.tv(o)
    #t.tv(o.wave)
    plt.plot(o.wave[10],o.data[10])
    #o.write(name + "_{:d}.fits".format(i))

(39, 4096)
[[9346.39929445 9345.38033199 9344.36130072 ... 5270.31717077
  5269.50846064 5268.69992212]
 [9346.33225011 9345.31369946 9344.29507923 ... 5271.7866107
  5270.97977298 5270.17310834]
 [9346.26867895 9345.25052108 9344.2322929  ... 5273.1894261
  5272.38437526 5271.57949893]
 ...
 [9346.17380282 9345.15767601 9344.14147499 ... 5281.24354708
  5280.44837117 5279.65337749]
 [9346.23179303 9345.21539291 9344.19891909 ... 5280.31450351
  5279.51812159 5278.72192094]
 [9346.29325642 9345.27656397 9344.25979835 ... 5279.31883539
  5278.52116195 5277.72366875]]
SEG3G2
(39, 4096)
[[9720.03331273 9719.00655192 9717.97972816 ... 5601.51788603
  5600.68166943 5599.84560894]
 [9720.09882412 9719.0719412  9718.04499547 ... 5601.4580115
  5600.62164384 5599.78543213]
 [9720.16129665 9719.13429767 9718.10723602 ... 5601.40152096
  5600.56500923 5599.72865331]
 ...
 [9720.4772423  9719.44989576 9718.42248716 ... 5601.49431707
  5600.65707155 5599.81998102]
 [9720.43335481 9719.40610388 971

In [ ]:
#trace2 = spectra.Trace(transpose=False)
#trace2.rows = [0,starec2[i].data.shape[0]]
#trace2.index = [0]

In [ ]:
#vars(trace2)

In [31]:
importlib.reload(spectra)
def model(x) :
    return x*0.

fig=plt.figure()
for i in range(len(starec2)) :
    #fig = plt.figure() 
    trace1 = spectra.Trace(transpose=False)
    trace1.rows = [0,starec2[i].data.shape[0]]
    trace1.index = [0]
    peak,ind = trace1.findpeak(starec2[i],thresh=10,sort=True)
    if len(peak) > 0:
        def model(x) :
            return x*0. + peak[0]
        trace1.model = [model]
        spec=trace1.extract(starec2[i],rad=5,back=[[-10,-5],[10,5]], display=t) # This line is important because you are extracting sky here.
        plt.figure(fig)
        spec.wave = starec2[i].wave[peak]
        print(spec.wave[0].shape,spec.data[0].shape)
        plt.plot(spec.wave[0],spec.data[0])
    else :
        print('no peak found for slit: ',i)
    #plt.draw()

looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20, 7]
aperture/fiber:  [1, 0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [18]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [19, 37, 6]
aperture/fiber:  [1, 2, 0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [21, 5, 37]
aperture/fiber:  [1, 0, 2]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [16, 36]
aperture/fiber:  [0, 1]
  extracting ... 
  See extraction window(s). Hit space bar

In [32]:
import copy
importlib.reload(spectra)
def model(x) :
    return x*0.

fig=plt.figure()
for i in range(len(starec2)) :
    trace3 = spectra.Trace(transpose=False)
    trace3.rows = [0,starec2[i].data.shape[0]]
    trace3.index = [0]
    peak,ind = trace3.findpeak(starec2[i],thresh=10,sort=True)
    if len(peak) > 0:
        def model(x) :
            return x*0. + peak[0]
        trace3.model = [model]
        spec=trace3.extract(starec2[i],rad=4, display=None) #,display=t) # back=[[-10,-5],[5,10]
        plt.figure(fig)
        spec.wave = starec2[i].wave[peak]
        swav=copy.deepcopy(wav)
        swav.skyline( spec, thresh=0.5 , linear= False, inter=plotinter, file='pyvista/data/sky/skyline.dat')
        print(wav.model)
        print(swav.model)
        
        #print(spec.model)
        #print(spec.wave[0].shape,spec.data[0].shape)
        #plt.plot(spec.wave[0],spec.data[0])
    else :
        print('no peak found for slit: ',i)
    #plt.draw()

looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20, 7]
aperture/fiber:  [1, 0]
  extracting ... 

  See identified lines.
  rms:    7.431
rejecting 1 points from 21 total: 
  rms:    1.948
rejecting 1 points from 21 total: 
  See 2D wavecal fit. Enter space in plot window to continue

Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0               c1_0        ...          c3_2        
    ----------------- ------------------- ... ---------------------
    7514.815916828107 -2102.6373855180495 ... -3.87896175372759e-05
Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0                c1_0        ...          c3_2        
    ------------------ ------------------- ... ---------------------
    2379.2719570058666 -2102.6373855180495 ... -3

C:\Users\abdul\pyvista\python\pyvista\spectra.py:695: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:    0.672
rejecting 1 points from 19 total: 
  rms:    0.436
rejecting 1 points from 19 total: 
  See 2D wavecal fit. Enter space in plot window to continue

Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0               c1_0        ...          c3_2        
    ----------------- ------------------- ... ---------------------
    7514.815916828107 -2102.6373855180495 ... -3.87896175372759e-05
Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0               c1_0        ...          c3_2        
    ----------------- ------------------- ... ---------------------
    2496.715624183644 -2102.6373855180495 ... -3.87896175372759e-05
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [19, 37, 6]
aperture/fiber:  [1, 2

C:\Users\abdul\pyvista\python\pyvista\spectra.py:695: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:    1.297
rejecting 0 points from 19 total: 
  See 2D wavecal fit. Enter space in plot window to continue

Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0               c1_0        ...          c3_2        
    ----------------- ------------------- ... ---------------------
    7514.815916828107 -2102.6373855180495 ... -3.87896175372759e-05
Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0                c1_0        ...          c3_2        
    ------------------ ------------------- ... ---------------------
    2499.2141371814296 -2102.6373855180495 ... -3.87896175372759e-05
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [21, 5, 37]
aperture/fiber:  [1, 0, 2]
  extracting ... 

  See identified lines.


C:\Users\abdul\pyvista\python\pyvista\spectra.py:695: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:    1.886
rejecting 0 points from 18 total: 
  See 2D wavecal fit. Enter space in plot window to continue

Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0               c1_0        ...          c3_2        
    ----------------- ------------------- ... ---------------------
    7514.815916828107 -2102.6373855180495 ... -3.87896175372759e-05
Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0                c1_0        ...          c3_2        
    ------------------ ------------------- ... ---------------------
    2504.5347838286852 -2102.6373855180495 ... -3.87896175372759e-05
